## Set Up Spark Session for Delta Lake on MinIO

This cell configures a SparkSession to work with **Delta Lake** using files stored on **MinIO** (S3-compatible storage).  
The setup includes:

- **Delta Lake support** via `delta-spark` package  
- **S3A configuration** to connect to MinIO with access keys  
- **Hadoop AWS** and **AWS SDK bundle** for S3 integration

The `.config(...)` options enable Delta SQL features, register the Delta catalog, and tell Spark how to connect to MinIO using the S3A protocol.



In [1]:
from pyspark.sql import SparkSession

packages = ",".join([
    "io.delta:delta-spark_2.12:3.2.0",       
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "com.amazonaws:aws-java-sdk-bundle:1.12.530",
])

# Create SparkSession with Delta and MinIO support
spark = (
    SparkSession.builder
      .appName("Delta Lakehouse on MinIO")
      .config("spark.jars.packages", packages)
      # Enable Delta Lake SQL support
      .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
      .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
      # Configure S3A (MinIO) access
      .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
      .config("spark.hadoop.fs.s3a.path.style.access", "true") # Enable path-style access (important for MinIO)
      .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") # Disable SSL for local setup
      .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
      .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
      .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
      .getOrCreate()
)
print("Spark:", spark.version)


Spark: 3.5.3


## Create Delta Table from Parquet Files

This step creates a new Delta Lake table `nyc.taxis_delta` in the `nyc` database.

- The table is created using the `delta` format.
- Data is loaded from two Parquet files stored in MinIO.
- The table is saved to the Delta-compatible path: `s3a://my-bucket/delta-lakehouse/nyc/taxis`.


In [2]:

spark.sql("CREATE DATABASE IF NOT EXISTS nyc")

spark.sql("""
CREATE TABLE IF NOT EXISTS nyc.taxis_delta
USING delta
LOCATION 's3a://my-bucket/delta-lakehouse/nyc/taxis'
AS
SELECT * FROM parquet.`s3a://my-bucket/raw-files/yellow_tripdata_2025-05.parquet`
UNION ALL
SELECT * FROM parquet.`s3a://my-bucket/raw-files/yellow_tripdata_2025-06.parquet`
""")


DataFrame[]

## Verify Row Count in Delta Table

This query checks how many rows were successfully loaded into the `nyc.taxis_delta` table.  
It confirms that data from both Parquet files was ingested correctly.

In [3]:
spark.sql("SELECT COUNT(*) FROM nyc.taxis_delta").show()


+--------+
|count(1)|
+--------+
| 8914805|
+--------+



## Inspect Delta Table Metadata and History

The first query (`DESCRIBE DETAIL`) shows metadata about the `nyc.taxis_delta` table, such as location, format, size, schema, and creation time.

The second query (`DESCRIBE HISTORY`) displays the table's commit history — including operations like `CREATE`, `WRITE`, or `MERGE`, along with timestamps and user information.



In [4]:
spark.sql(""" 
DESCRIBE DETAIL nyc.taxis_delta;
""").show()

spark.sql(""" 
DESCRIBE HISTORY nyc.taxis_delta;
""").show()

+------+--------------------+--------------------+-----------+--------------------+--------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+--------------+
|format|                  id|                name|description|            location|           createdAt|       lastModified|partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion| tableFeatures|
+------+--------------------+--------------------+-----------+--------------------+--------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+--------------+
| delta|f7e66181-03b1-4bd...|spark_catalog.nyc...|       NULL|s3a://my-bucket/d...|2025-08-10 16:18:...|2025-08-10 16:18:26|              []|               []|      10|  183028768|        {}|               3|               7|[timestampNtz]|
+------+--------------------+-------

## Simulate New Batch Ingestion and Track Changes

A small random sample (~10%) from the June Parquet file is inserted into the `nyc.taxis_delta` table to simulate a new batch of data.

After the insert, `DESCRIBE HISTORY` is used again to verify that a new commit was recorded.  
This allows tracking how and when the table was updated — a core feature of Delta Lake's transaction log.


In [5]:

spark.sql("""INSERT INTO nyc.taxis_delta
SELECT * FROM parquet.`s3a://my-bucket/raw-files/yellow_tripdata_2025-06.parquet`
WHERE rand() < 0.10;
""")

spark.sql("""DESCRIBE HISTORY nyc.taxis_delta;
""").show()



+-------+-------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2025-08-10 16:18:29|  NULL|    NULL|               WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          0|  Serializable|         true|{numFiles -> 6, n...|        NULL|Apache-Spark/3.5....|
|      0|2025-08-10 16:18:26|  NULL|    NULL|CREATE TABLE AS S...|{partitionBy -> [...|NULL|    NULL|     NULL|       NULL|  Serializable|         true|{numFiles -> 11,

## Query Delta Table at Specific Versions

These queries read the `nyc.taxis_delta` table as it existed at earlier points in time using Delta Lake’s **time travel** feature:

- `VERSION AS OF 0` returns the original state of the table (after the initial load).
- `VERSION AS OF 1` includes the additional rows from the simulated batch insert.


In [6]:
spark.sql(""" 
SELECT COUNT(*) 
FROM delta.`s3a://my-bucket/delta-lakehouse/nyc/taxis` VERSION AS OF 0;""").show()

spark.sql(""" 
SELECT COUNT(*) 
FROM delta.`s3a://my-bucket/delta-lakehouse/nyc/taxis` VERSION AS OF 1;""").show()



+--------+
|count(1)|
+--------+
| 8914805|
+--------+

+--------+
|count(1)|
+--------+
| 9346736|
+--------+



## Restore Delta Table to an Earlier Version

This command rolls back the `nyc.taxis_delta` table to version 0 — effectively undoing the last insert.
The `DESCRIBE HISTORY` query then shows a new entry for the `RESTORE` operation, confirming that the rollback was recorded in the Delta transaction log.


In [7]:
spark.sql(""" 
RESTORE TABLE nyc.taxis_delta TO VERSION AS OF 0;""")

spark.sql(""" 
DESCRIBE HISTORY nyc.taxis_delta;""").show()

+-------+-------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      2|2025-08-10 16:18:33|  NULL|    NULL|             RESTORE|{version -> 0, ti...|NULL|    NULL|     NULL|          1|  Serializable|        false|{numRestoredFiles...|        NULL|Apache-Spark/3.5....|
|      1|2025-08-10 16:18:29|  NULL|    NULL|               WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          0|  Serializable|         true|{numFiles -> 6, 

## Delta Lake Change Data Feed (CDF) Demo

This sequence shows how to enable and use CDF on `nyc.taxis_delta`:

1. **Enable CDF**  
   Turn on `delta.enableChangeDataFeed = true` so future commits record row-level changes.

2. **Mark the starting version**  
   Read `DESCRIBE HISTORY` and capture the current table version before making changes.

3. **Make controlled changes**  
   - `INSERT` a tiny, deterministic slice (one day, small sample).  
   - `UPDATE` a subset (e.g., set `tip_amount = 0` for a day/vendor filter).  
   - `DELETE` a few rows (e.g., very short trips that day).

4. **Read only the changes**  
   Load the table with:
   - `readChangeFeed = true`  
   - `startingVersion = <captured version>`  
   - `endingVersion = latest`  
   This returns only changed rows between those versions.

5. **Inspect change types**  
   CDF adds metadata columns:
   - `_change_type`: `insert`, `update_preimage`, `update_postimage`, `delete`  
   - `_commit_version`, `_commit_timestamp`  
   Select relevant business columns plus these fields to see exactly what changed.  
   (Optional) `GROUP BY _change_type` to summarize counts.

Notes:
- CDF starts recording **after** it’s enabled; earlier versions won’t have change rows.
- If filters affect few records, you may only see `insert` rows (or small counts) in CDF.


In [8]:
from pyspark.sql.functions import col

# 1) Enable CDF (applies to future commits)
spark.sql("""
ALTER TABLE nyc.taxis_delta
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

# 2) Capture the current version BEFORE changes
start_version = (spark.sql("DESCRIBE HISTORY nyc.taxis_delta")
                   .selectExpr("max(version) as v").collect()[0][0])

# 3) Make some changes: INSERT + UPDATE + DELETE
# Insert a small, deterministic slice (one day, tiny sample)
spark.sql("""
INSERT INTO nyc.taxis_delta
SELECT *
FROM parquet.`s3a://my-bucket/raw-files/yellow_tripdata_2025-06.parquet`
WHERE tpep_pickup_datetime >= '2025-06-15'
  AND tpep_pickup_datetime <  '2025-06-16'
  AND rand() < 0.02
""")

# Update a small subset (same day, common vendors)
spark.sql("""
UPDATE nyc.taxis_delta
SET tip_amount = 0.0
WHERE date(tpep_pickup_datetime) = '2025-06-15' AND VendorID IN (1,2)
""")

# Delete a tiny subset from that day (very short trips)
spark.sql("""
DELETE FROM nyc.taxis_delta
WHERE date(tpep_pickup_datetime) = '2025-06-15' AND trip_distance < 0.1
""")

# 4) Read just the changes between start_version -> latest
end_version = (spark.sql("DESCRIBE HISTORY nyc.taxis_delta")
                 .selectExpr("max(version) as v").collect()[0][0])

cdf = (spark.read.format("delta")
       .option("readChangeFeed", "true")
       .option("startingVersion", start_version)
       .option("endingVersion", end_version)
       .table("nyc.taxis_delta"))

# 5) Inspect change types
cdf.select("_commit_version", "_commit_timestamp", "_change_type",
           "VendorID", "tpep_pickup_datetime", "tip_amount", "total_amount") \
   .show(truncate=False)



+---------------+----------------------+----------------+--------+--------------------+----------+------------+
|_commit_version|_commit_timestamp     |_change_type    |VendorID|tpep_pickup_datetime|tip_amount|total_amount|
+---------------+----------------------+----------------+--------+--------------------+----------+------------+
|5              |2025-08-10 16:18:39.37|update_preimage |2       |2025-06-15 00:02:13 |3.29      |19.74       |
|5              |2025-08-10 16:18:39.37|update_postimage|2       |2025-06-15 00:02:13 |0.0       |19.74       |
|5              |2025-08-10 16:18:39.37|update_preimage |2       |2025-06-15 00:44:11 |3.45      |14.95       |
|5              |2025-08-10 16:18:39.37|update_postimage|2       |2025-06-15 00:44:11 |0.0       |14.95       |
|5              |2025-08-10 16:18:39.37|update_preimage |1       |2025-06-15 00:16:04 |0.0       |68.85       |
|5              |2025-08-10 16:18:39.37|update_postimage|1       |2025-06-15 00:16:04 |0.0       |68.85 

In [9]:
cdf.groupBy("_change_type").count().show()

+----------------+------+
|    _change_type| count|
+----------------+------+
| update_preimage|125545|
|update_postimage|125545|
|          delete|  2088|
|          insert|  2493|
+----------------+------+



# Add and Populate a New Column in Delta Table

This step adds a new column `fare_bucket` to the `nyc.taxis_delta` table and fills it with values based on `total_amount`:

- `'low'` for fares under 10  
- `'mid'` for fares between 10 and 30  
- `'high'` for fares 30 and above

The final query groups the data by `fare_bucket` to show the distribution of fare ranges.


In [10]:
# Disabling CDF because of Java OOM
spark.sql("""ALTER TABLE nyc.taxis_delta UNSET TBLPROPERTIES (delta.enableChangeDataFeed);""")

spark.sql("""ALTER TABLE nyc.taxis_delta ADD COLUMNS (fare_bucket STRING);""")

spark.sql("""UPDATE nyc.taxis_delta
SET fare_bucket = CASE
  WHEN total_amount < 10 THEN 'low'
  WHEN total_amount < 30 THEN 'mid'
  ELSE 'high'
END
WHERE total_amount IS NOT NULL;""")

spark.sql("""SELECT fare_bucket, COUNT(*) 
FROM nyc.taxis_delta 
GROUP BY fare_bucket;""").show()

+-----------+--------+
|fare_bucket|count(1)|
+-----------+--------+
|        low|  765108|
|        mid| 5744522|
|       high| 2405580|
+-----------+--------+



## Partition Evolution in Delta Lake

Delta Lake does not support partition evolution.  
Once a table’s partitioning is defined at creation, it cannot be changed without recreating the table and reloading the data.
